In [1099]:
import argparse
import bz2
import xml.etree.ElementTree as ET
import pandas as pd
import pickle
import category_encoders as ce

In [1112]:
def get_page(wiki_file):

    page_lines = []
    page_started, page_ended = False, False
    
    #print(wiki_file)

    while not page_started:
        line = wiki_file.readline()
        if line == '':
            return False
        if line.find('<page>') != -1:
            page_started = True
            page_lines.append(line)

    while not page_ended:
        line = wiki_file.readline()
        if line.find('</page>') != -1:
            page_ended = True
        page_lines.append(line)

    return ''.join(page_lines)


def parse_wiki(input_path):
    import_data_pd = pd.DataFrame()
    
    wiki_file = bz2.open(input_path,mode='t',encoding="UTF8")

    category_list = []
    text_list = []
    company_temp_list = []
    count = 0
    
    while True:
        if count%10000 == 0:
            print(count+len(company_temp_list))
            if len(company_temp_list) > 10000:
                break
        
        page_str = get_page(wiki_file)
        #print(page_str)
        if not page_str:
            break

        # XMLをパースする
        root = ET.fromstring(page_str)

        title = root.find('title').text
        
        #print(title)
        gaiyou_str,category_str = get_gaiyou(page_str)
        
        if gaiyou_str != "temp" and category_str != "temp" and gaiyou_str.strip() != "":
            gaiyou_str_cleaned = clean_text(gaiyou_str)
            if gaiyou_str_cleaned.strip()[:5:] != "file:" and gaiyou_str_cleaned.strip()[:3:] != "画像:" and gaiyou_str_cleaned.strip()[:5:] != "ファイル:":
                

                #print(gaiyou_str)
                category_list.append(int(category_str))
                text_list.append(gaiyou_str_cleaned)
                company_temp_list.append(title)
            
        count += 1
                
    import_data_pd["category"] = category_list
    import_data_pd["text"] = text_list
    import_data_pd["company"] = company_temp_list
    
    return import_data_pd
                
                
def get_gaiyou(page_str):
    page_lines = []
    company_flag, gaiyou_start, gaiyou_end = False, False, False
    gaiyou_str = "temp"
    category_str = "temp"

    if page_str.find('|社名') != -1 or page_str.find('| 社名') != -1:
        
        category_num = page_str.find('|業種')
        category_num1 = page_str.find('| 業種')
        if  category_num != -1 or category_num1 != -1:
            
            if  category_num != -1:
                temp_str = page_str[category_num+3::]
            elif  category_num1 != -1:
                temp_str = page_str[category_num1+4::]
            
            category_temp_str = temp_str[temp_str.find('=')+1:temp_str.find('|'):].strip()
            
            category_temp_str = re.sub(r'[\(（].*?[\)）]', '', category_temp_str).strip()
            #print(category_temp_str)
            
            #カテゴリが文字列の場合
            if not re.match(r"^[0-9]{4}$", category_temp_str):
                if len(category_df["category"][category_df["中分類"] == category_temp_str]) != 0:
                    category_temp_str = category_df["category"][category_df["中分類"] == category_temp_str].values[0]
                
            if re.match(r"^[0-9]{4}$", category_temp_str):
                category_str = category_temp_str
            
            num = page_str.find('== 概要 ==')
            num1 = page_str.find('==概要==')
            num2 = page_str.find('== 概説 ==')
            num3 = page_str.find('==概説==')
        
            if num != -1:
                temp_str = page_str[num+8::]
                gaiyou_str = temp_str[:temp_str.find('=='):]
            elif num1 != -1:
                temp_str = page_str[num1+6::]
                gaiyou_str = temp_str[:temp_str.find('=='):]
            elif num2 != -1:
                temp_str = page_str[num2+8::]
                gaiyou_str = temp_str[:temp_str.find('=='):]
            elif num3 != -1:
                temp_str = page_str[num3+6::]
                gaiyou_str = temp_str[:temp_str.find('=='):]
                
    return gaiyou_str,category_str

In [1113]:
category_df = pd.read_csv("category.csv",encoding="SJIS",sep="\t",dtype = {"category":"object"})
category_df

,中分類,category
0,水産・農林業,0050
1,鉱業,1050
2,建設業,2050
3,食料品,3050
4,繊維製品,3100
5,パルプ・紙,3150
6,化学,3200
7,医薬品,3250
8,石油・石炭製品,3300
9,ゴム製品,3350


In [1114]:
company_df = pd.read_csv("company.csv",encoding="UTF8",sep="\t",dtype={"category":"object"})
company_df

,company,category
0,アクシーズ,0050
1,アグリ・ヴァンティアン,0050
2,カネコ種苗,0050
3,サカタのタネ,0050
4,ジャスダック,0050
5,ベルグアース,0050
6,ホクト,0050
7,ホクリヨウ,0050
8,ホーブ,0050
9,マルハ,0050


In [1115]:
#company_list = list(company_pd["company"])
wiki_file = "jawiki-latest-pages-articles.xml.bz2"

import_data_pd = parse_wiki(wiki_file)

0
10063
20137
30228
40344
50475
60649
70717
80786
90881
100977
111048
121121
131212
141275
151322
161382
171450
181515
191569
201626
211686
221755
231806
241867
251917
261968
272012
282078
292126
302176
312242
322324
332384
342446
352523
362594
372652
382704
392744
402793
412831
422879
432921
442958
452997
463055
473100
483190
493247
503288
513358
523428
533501
543572
553650
563705
573755
583796
593837
603870
613917
623984
634024
644060
654094
664118
674151
684191
694236
704271
714311
724347
734381
744425
754458
764495
774534
784622
794663
804713
814766
824796
834827
844870
854911
864949
874978
885007
895035
905077
915102
925144
935182
945225
955253
965296
975335
985366
995402
1005442
1015479
1025536
1035586
1045617
1055645
1065678
1075715
1085754
1095801
1105845
1115887
1125927
1135963
1145996
1156056
1166091
1176123
1186172
1196227
1206281
1216310
1226370
1236420
1246459
1256495
1266545
1276579
1286618
1296660
1306699
1316746
1326787
1336813
1346861
1356905
1366940
1376984
1387014
13

In [1116]:
#category_odの作成
oe = ce.OrdinalEncoder(cols="category", handle_unknown='ignore')
oe.fit(category_df["category"])
import_data_pd_oe = oe.transform(import_data_pd["category"])
import_data_pd["category_od"] = import_data_pd_oe["category"] - 1

In [1117]:
#category_odの整理
import_data_pd = import_data_pd.drop(import_data_pd[import_data_pd["category_od"].isnull()].index,axis=0)
import_data_pd["category_od"] = import_data_pd["category_od"].astype(int)

In [1118]:
#保存
pickle.dump(import_data_pd,open("import_data_pd.sav","wb"))
import_data_pd.to_csv("import_data_pd.csv")

import_data_pd

,category,text,company,category_od
0,9050,一軍は大阪府をプロ野球地域保護権保護地域とし、大阪市西区大阪市西区にある大阪ドーム現：京セラ...,大阪近鉄バファローズ,32
1,5050,京浜急行電鉄は、東京都港区東京都港区から品川区、大田区、神奈川県川崎市、横浜市、さらに三浦半...,京浜急行電鉄,20
2,5250,1969年3月21日に大阪府豊中市で創業して1973年3月19日に法人化された。ジュークボッ...,コナミホールディングス,24
3,3650,理化学研究所と共同開発したスーパーコンピュータ「京スーパーコンピュータ京けい」が、計算速度の...,富士通,15
4,5250,1975年9月、福嶋康博により「営団社募集サービスセンター」として創業。紆余曲折を経て、19...,エニックス,24
5,5250,1955年6月1日に中村雅哉が有限会社中村製作所なかむらせいさくじょを設立。1971年に「」...,バンダイナムコエンターテインメント,24
6,3650,事業内容はコンピュータ関連のサービスおよびコンサルティングの提供と、ソフトウェア、ハードウェ...,IBM,15
7,3650,主にマイクロプロセッサ、チップセット、フラッシュメモリなどの設計開発・製造・販売を手掛けてい...,インテル,15
8,5250,1973年5月18日、工藤裕司現在のペンネームは三遊亭あほまろとその弟工藤浩により札幌市豊平...,ハドソン,24
9,5250,x68000向けゲームから始まり、スーパーファミコンのドラッキーシリーズ発売:イマジニアやゲ...,ズーム (ゲーム会社),24


In [1028]:
#読み込み
import_data_pd = pickle.load(open("import_data_pd.sav","rb"))

In [638]:
# 各Layerの設定
import numpy as np

class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
        return None

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        h_next = np.tanh(t)
        self.cache = (x, h_prev, h_next)
        return h_next

    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache
        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis=0)
        dWh = np.dot(h_prev.T, dt)
        dh_prev = np.dot(dt, Wh.T)
        dWx = np.dot(x.T, dt)
        dx = np.dot(dt, Wx.T)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev

class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x):
        W, b = self.params
        out = np.dot(x, W) + b
        self.x = x
        return out

    def backward(self, dout):
        W, b = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0)

        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

In [639]:
# クラス分類層と損失関数の定義
# 分類機はSoftmaxなので、他クラス分類の場合はoutput_sizeをクラス数に変更するだけで良いです。

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    if t.size == y.size:
        t = t.argmax(axis=1)
    batch_size = y.shape[0]
    result = -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
    return result

def softmax(x):
    if x.ndim == 2:
        x = x - x.max(axis=1, keepdims=True)
        x = np.exp(x)
        x /= x.sum(axis=1, keepdims=True)
    elif x.ndim == 1:
        x = x - np.max(x)
        x = np.exp(x) / np.sum(np.exp(x))
    return x

class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y= None  # softmaxの出力
        self.t = None  # 教師ラベル

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)

        # 教師ラベルがone-hotベクトルの場合、正解のインデックスに変換
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)
        loss = cross_entropy_error(self.y, self.t)
        return loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1
        dx *= dout
        dx = dx / batch_size

        return dx

In [640]:
class RnnModel:
    def __init__(self, vocab_size, embed_size, hidden_size, out_size):
        # クラスの初期化
        # :param vocab_size: 単語数
        # :param embed_size: 埋め込みベクトルサイズ
        # :param hidden_size: 隠れ層サイズ
        # :param out_size: 出力層サイズ
        V, E, H, O = vocab_size, embed_size, hidden_size, out_size
        rn = np.random.randn

        self.H = H

        self.embed_W = (rn(V,E) / 100).astype("f")
        self.rnn_Wx     = (rn(E,H) / np.sqrt(E)).astype("f")
        self.rnn_Wh     = (rn(H,H) / np.sqrt(H)).astype("f")
        self.rnn_b        = np.zeros(H).astype("f")
        self.affine_W   = (rn(H,O) / np.sqrt(H)).astype("f")
        self.affine_b    = np.zeros(O).astype("f")
        # 内部状態行列
        self.h = None

        self.layers = [
            Embedding(self.embed_W),
            RNN(self.rnn_Wx, self.rnn_Wh, self.rnn_b),
            Affine(self.affine_W, self.affine_b)
        ]
        self.embedding_layer = self.layers[0]
        self.rnn_layer = self.layers[1]
        self.affine_layer = self.layers[2]
        self.loss_layer = SoftmaxWithLoss()

        self.params, self.grads = [],[]
        for layer in self.layers:
            self.params += layer.params
            self.grads  += layer.grads

    def predict(self, x):
        N,T = x.shape
        H = self.H
        x = x.T
        self.h = np.zeros((N,H), dtype="f")
        for time, word in enumerate(x):
            e = self.embedding_layer.forward(word)
            rnn_layer = RNN(self.rnn_Wx, self.rnn_Wh, self.rnn_b)
            self.h = rnn_layer.forward(e, self.h)
        y = self.affine_layer.forward(self.h)
        y = softmax(y)
        return y

    def forward(self, x, t):
        N, T = x.shape
        H = self.H
        self.x = x
        x = x.T
        self.layers = []
        self.hs = np.empty((N,T,H), dtype="f")
        self.h = np.zeros((N,H), dtype="f")
        for time, word in enumerate(x):
            e = self.embedding_layer.forward(word)
            layer = RNN(self.rnn_Wx, self.rnn_Wh, self.rnn_b)
            self.h = layer.forward(e, self.h)
            self.hs[:,time,:] = self.h
            self.layers.append(layer)
        y = self.affine_layer.forward(self.h)
        loss = self.loss_layer.forward(y, t)
        return loss

    def backward(self, dh=1):
        dh = self.loss_layer.backward(dh)
        dh = self.affine_layer.backward(dh)
        x = self.x.T
        for i, word in enumerate(reversed(x)):
            time = len(x) - 1 - i
            layer = self.layers[time]
            dx, dh = layer.backward(dh)
        dx = self.embedding_layer.backward(dx)
        return dx

    def set_state(self, h):
        self.h = h

    def reset_state(self):
        self.h = None

In [641]:
class Adam:
  def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
    self.lr = lr
    self.beta1 = beta1
    self.beta2 = beta2
    self.iter = 0
    self.m = None
    self.v = None

  def update(self, params, grads):
    if self.m is None:
      self.m, self.v = [], []
      for param in params:
        self.m.append(np.zeros_like(param))
        self.v.append(np.zeros_like(param))

    self.iter += 1
    lr_t = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)

    for i in range(len(params)):
      self.m[i] += (1 - self.beta1) * (grads[i] - self.m[i])
      self.v[i] += (1 - self.beta2) * (grads[i]**2 - self.v[i])

      params[i] -= lr_t * self.m[i] / (np.sqrt(self.v[i]) + 1e-7)

In [642]:
from janome.tokenizer import Tokenizer
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *
jt = Tokenizer()


class NumericReplaceFilter():
    """
    名詞中の数(漢数字を含む)を全て0に置き換えるTokenFilterの実装
    
    Parameters
    ----------
    text : TokenFilter
        前処理する日本語の文字列

    Returns
    -------
    token : str
        前処理した日本語の文字列
    """
    def apply(self, tokens):
        for token in tokens:
            parts = token.part_of_speech.split(',')
            if (parts[0] == '名詞' and parts[1] == '数'):
                token.surface = '0'
                token.base_form = '0'
                token.reading = 'ゼロ'
                token.phonetic = 'ゼロ'
            yield token

In [1107]:
import re
import numpy as np

# テキストから不要な要素を取り除く

def clean_text(text):
    """
    日本語の文字列を前処理する

    Parameters
    ----------
    text : str
        前処理する日本語の文字列

    Returns
    -------
    cleand_text : str
        前処理した日本語の文字列
    """
    replaced_text = '\n'.join(s.strip() for s in text.splitlines() if s != '')  # skip header by [2:]
    replaced_text = replaced_text.lower()
    replaced_text = re.sub(r'[【】]', ' ', replaced_text)       # 【】の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去

    soup = BeautifulSoup(replaced_text, 'html.parser')
    cleaned_text = soup.get_text()
    cleaned_text = ''.join(cleaned_text.splitlines())
    
    cleaned_text = re.sub("'''", '', cleaned_text)
    cleaned_text = re.sub("\|", '', cleaned_text)
    cleaned_text = re.sub(r'\<ref.*?\<\/ref\>', '', cleaned_text)
    cleaned_text = re.sub(r'\{\{.*?\}\}', '', cleaned_text)
    cleaned_text = re.sub(r'\<.*?\>', '', cleaned_text)
    cleaned_text = re.sub(r' ', '', cleaned_text)
    cleaned_text = re.sub(r'　', '', cleaned_text)
    cleaned_text = re.sub(r'[a-z]{10,}', '', cleaned_text)
    
    return cleaned_text


def convert_sentence_to_words(sentence):
    #stopwords = ["i", "a", "an", "the", "and", "or", "if", "is", "are", "am", "it", "this", "that", "of", "from", "in", "on"]
    sentence = sentence.lower() # 小文字化
    sentence = sentence.replace("\n", "") # 改行削除
    sentence = re.sub(re.compile("[!-\/:-@[-`{-~]"), " ", sentence) # 記号をスペースに置き換える
    sentence = sentence.split(" ") # スペースで区切る
    sentence_words = []
    for word in sentence:
        if (re.compile(r"^.*[0-9]+.*$").fullmatch(word) is not None):
            continue
        #if word in stopwords:
        #    continue
        sentence_words.append(word)
    return sentence_words

def janome_preprocessing(text):
    """
    前処理と形態素解析をする

    Parameters
    ----------
    text : str
        前処理する日本語の文字列

    Returns
    -------
    text_list : list[str]
        形態素解析後の文字列のリスト
    """
    text_list = []
    char_filters = [UnicodeNormalizeCharFilter()]
    #token_filters = [NumericReplaceFilter(),POSStopFilter(['記号','助詞']), LowerCaseFilter()]
    token_filters = [NumericReplaceFilter(),LowerCaseFilter()]
    tokenizer = Tokenizer()
    a = Analyzer(char_filters, tokenizer, token_filters)
    
    for token in a.analyze(text):
        text_list.append(token.surface)
        
    return text_list

# 単語をIDに変換する単語辞書を生成する
def create_word_id_dict(sentence_list):
    word_to_id = {}
    for sentence in sentence_list:
        sentence_words = janome_preprocessing(sentence)
        for word in sentence_words:
            if word not in word_to_id:
                word_to_id[word] = len(word_to_id)
    return word_to_id

# 文章をID列に変換する
def convert_sentences_to_ids(sentence_list, word_to_id):
    sentence_id_vec = []
    for sentence in sentence_list:
        sentence_words = janome_preprocessing(sentence)
        sentence_ids = []
        for word in sentence_words:
            if word in word_to_id:
                sentence_ids.append(word_to_id[word])
            else:
                sentence_ids.append(-1)
        sentence_id_vec.append(sentence_ids)
    return sentence_id_vec

# 文章はそれぞれ長さが違うので、前パディングして固定長の系列にする
# 前パディングなのは、RNNの構造上、後ろにあるデータのほうが反映されやすいため
def padding_sentence(sentence_id_vec):
    max_sentence_size = 0
    for sentence_vec in sentence_id_vec:
        if max_sentence_size < len(sentence_vec):
            max_sentence_size = len(sentence_vec)
    for sentence_ids in sentence_id_vec:
        while len(sentence_ids) < max_sentence_size:
            sentence_ids.insert(0,-1)
    return sentence_id_vec

In [1014]:
data_list = import_data_pd[["text","category_od"]].values.tolist()
import_data_list = import_data_pd["text"].values.tolist()

word_to_id = create_word_id_dict(import_data_list)
print(1)
sentence_id_vec = convert_sentences_to_ids(import_data_list, word_to_id)
print(2)
sentence_id_vec = padding_sentence(sentence_id_vec)

1
2


In [1044]:
data_list = import_data_pd[["text","category_od"]].values.tolist()

In [1049]:
import_data_pd

,category,text,company,category_od
0,9050,一軍は 大阪府 を プロ野球地域保護権|保護地域 とし、 大阪市 西区 大...,大阪近鉄バファローズ,32
1,5050,京浜急行電鉄は、 東京都 港区 東京都 |港区 から 品川区 、 大田区 ...,京浜急行電鉄,20
2,5250,1969年3月21日に大阪府豊中市で創業して1973年3月19日に 法人 化された。 ...,コナミホールディングス,24
3,3650,理化学研究所 と共同開発した スーパーコンピュータ 「 京 スーパーコンピュー...,富士通,15
4,5250,1975年9月、 福嶋康博 により「営団社募集サービスセンター」として創業。紆余曲折を経...,エニックス,24
5,5250,1955年 6月1日 に 中村雅哉 が'''有限会社中村製作所''' なかむ...,バンダイナムコエンターテインメント,24
6,3650,事業内容は コンピュータ 関連の サービス および コンサルティング の提供と、...,IBM,15
7,3650,主に マイクロプロセッサ 、 チップセット 、 フラッシュメモリ などの設計開発...,インテル,15
8,5250,1973年 5月18日、工藤裕司 現在のペンネームは三遊亭あほまろ とその弟工藤浩によ...,ハドソン,24
9,5250,x68000 向けゲームから始まり、 スーパーファミコン のドラッキーシリーズ 発...,ズーム (ゲーム会社),24


In [691]:
text = '2000年  に  第二電電    ddi 、  ケイディディ    kdd 、  日本移動通信    ido  が  合併  企業 |合併  し、「\'\'\'株式会社ディーディーアイ\'\'\'」 通称及びロゴマークはkddi として発足する<ref> 沿革  kddiについて  - kddi株式会社、2016年1月7日閲覧。</ref><ref> 合併契約について  - 第二電電株式会社  kdd株式会社  日本移動通信株式会社 2000年4月5日ニュースリリース 2002年2月22日時点での  インターネットアーカイブ   、2016年1月7日閲覧。</ref><ref> 合併認可申請について〔別紙〕  - 第二電電株式会社  kdd株式会社  日本移動通信株式会社 2000年8月9日ニュースリリース 2002年3月11日時点でのインターネットアーカイブ 、2016年1月7日閲覧。</ref><ref> 公正取引委員会、第二電電㈱，ケイディディ 株 及び日本移動通信 株 の合併 新会社名 kddi㈱  </ref>。2001年4月に合併時より通称として使用していた「kddi」を社名とし「\'\'\'ケイディーディーアイ株式会社\'\'\'<ref name="kddi">対外的表記は2001年4月より「\'\'\'kddi株式会社\'\'\'」を使用していたが、当時はアルファベットが  商業登記|登記  上使用できなかったためこの表記となった。詳しくは  商号#商号登記  の項目を参照。</ref>」に社名変更し<ref name="shikiho2002shinshun">「業種・社名変更会社一覧」、『  会社四季報  』2002年1集新春号、  東洋経済新報社  、2001年、43頁。2016年1月23日閲覧。</ref>、2002年11月に現社名の「\'\'\'kddi株式会社\'\'\'」に変更。'
text_list = []

a = Analyzer(char_filters, tokenizer, token_filters)

for token in a.analyze(text):
    text_list.append(token.surface)
clean_text(''.join(text_list))



"0年  に  第二電電    ddi 、  ケイディディ    kdd 、  日本移動通信    ido  が  合併  企業 |合併  し、「'''株式会社ディーディーアイ'''」 通称及びロゴマークはkddi として発足する 沿革  kddiについて  - kddi株式会社、0年0月0日閲覧。 合併契約について  - 第二電電株式会社  kdd株式会社  日本移動通信株式会社 0年0月0日ニュースリリース 0年0月0日時点での  インターネットアーカイブ   、0年0月0日閲覧。 合併認可申請について〔別紙〕  - 第二電電株式会社  kdd株式会社  日本移動通信株式会社 0年0月0日ニュースリリース 0年0月0日時点でのインターネットアーカイブ 、0年0月0日閲覧。 公正取引委員会、第二電電 株 ,ケイディディ 株 及び日本移動通信 株 の合併 新会社名 kddi 株   。0年0月に合併時より通称として使用していた「kddi」を社名とし「'''ケイディーディーアイ株式会社'''対外的表記は0年0月より「'''kddi株式会社'''」を使用していたが、当時はアルファベットが  商業登記|登記  上使用できなかったためこの表記となった。詳しくは  商号#商号登記  の項目を参照。」に社名変更し「業種・社名変更会社一覧」、『  会社四季報  』0年0集新春号、  東洋経済新報社  、0年、0頁。0年0月0日閲覧。、0年0月に現社名の「'''kddi株式会社'''」に変更。"

In [712]:
janome_preprocessing(text)

['0',
 '年',
 '第二電電',
 'ddi',
 'ケイディディ',
 'kdd',
 '日本移動通信',
 'ido',
 'が',
 '合併',
 '企業',
 '|',
 '合併',
 'し',
 "「'''",
 '株式会社',
 'ディーディーアイ',
 "'''」",
 '通称',
 '及び',
 'ロゴ',
 'マーク',
 'kddi',
 '発足',
 'する',
 '<',
 'ref',
 '>',
 '沿革',
 'kddi',
 '-',
 'kddi',
 '株式会社',
 '0',
 '年',
 '0',
 '月',
 '0',
 '日',
 '閲覧',
 '</',
 'ref',
 '><',
 'ref',
 '>',
 '合併',
 '契約',
 '-',
 '第二電電',
 '株式会社',
 'kdd',
 '株式会社',
 '日本移動通信',
 '株式会社',
 '0',
 '年',
 '0',
 '月',
 '0',
 '日',
 'ニュース',
 'リリース',
 '0',
 '年',
 '0',
 '月',
 '0',
 '日',
 '時点',
 'インターネット',
 'アーカイブ',
 '0',
 '年',
 '0',
 '月',
 '0',
 '日',
 '閲覧',
 '</',
 'ref',
 '><',
 'ref',
 '>',
 '合併',
 '認可',
 '申請',
 '別紙',
 '-',
 '第二電電',
 '株式会社',
 'kdd',
 '株式会社',
 '日本移動通信',
 '株式会社',
 '0',
 '年',
 '0',
 '月',
 '0',
 '日',
 'ニュース',
 'リリース',
 '0',
 '年',
 '0',
 '月',
 '0',
 '日',
 '時点',
 'インターネット',
 'アーカイブ',
 '0',
 '年',
 '0',
 '月',
 '0',
 '日',
 '閲覧',
 '</',
 'ref',
 '><',
 'ref',
 '>',
 '公正',
 '取引',
 '委員',
 '会',
 '第二電電',
 '(',
 '株',
 '),',
 'ケイディディ',
 '株',
 '及び',
 '日本移動通信',
 '株',
 '合併',


In [1050]:
# ハイパーパラメータの設定
vocab_size = len(word_to_id)
embed_size = 100
hidden_size = 50
out_size = 33
lr = 1e-3
batch_size = 37
max_epochs = 1
data_size = len(import_data_list)
max_iters = data_size // batch_size

model = RnnModel(vocab_size, embed_size, hidden_size, out_size)
optimizer = Adam(lr)

In [1051]:
# Numpy配列に変換する必要がある
x = np.array(sentence_id_vec)
t = np.array([label[1] for label in data_list])

total_loss = 0
loss_count = 0
for epoch in range(max_epochs):
    for iters in range(max_iters):
        batch_x  = x[iters*batch_size:(iters+1)*batch_size]
        batch_t =  t[iters*batch_size:(iters+1)*batch_size]
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)

        total_loss += loss
        loss_count += 1

        if (iters) % 10 == 0:
            avg_loss = total_loss / loss_count
            print("| epoch %d | iter %d / %d | loss %.5f" % (epoch+1, iters, max_iters, avg_loss))
            total_loss, loss_count = 0,0

| epoch 1 | iter 0 / 262 | loss 3.44041
| epoch 1 | iter 10 / 262 | loss 3.09052
| epoch 1 | iter 20 / 262 | loss 3.01621
| epoch 1 | iter 30 / 262 | loss 3.11187
| epoch 1 | iter 40 / 262 | loss 3.05619
| epoch 1 | iter 50 / 262 | loss 3.02527
| epoch 1 | iter 60 / 262 | loss 2.97186
| epoch 1 | iter 70 / 262 | loss 3.00203
| epoch 1 | iter 80 / 262 | loss 2.81010
| epoch 1 | iter 90 / 262 | loss 3.01252
| epoch 1 | iter 100 / 262 | loss 2.95577
| epoch 1 | iter 110 / 262 | loss 2.79849
| epoch 1 | iter 120 / 262 | loss 2.78594
| epoch 1 | iter 130 / 262 | loss 2.68186
| epoch 1 | iter 140 / 262 | loss 2.79431
| epoch 1 | iter 150 / 262 | loss 2.73585
| epoch 1 | iter 160 / 262 | loss 2.62660
| epoch 1 | iter 170 / 262 | loss 2.63704
| epoch 1 | iter 180 / 262 | loss 2.74249
| epoch 1 | iter 190 / 262 | loss 2.74707
| epoch 1 | iter 200 / 262 | loss 2.74374
| epoch 1 | iter 210 / 262 | loss 2.66309
| epoch 1 | iter 220 / 262 | loss 2.67971
| epoch 1 | iter 230 / 262 | loss 2.54011
| e

In [1062]:
batch_x

array([[   -1,    -1,    -1, ...,    45,   189,    47],
       [   -1,    -1,    -1, ...,   190,   155,   267],
       [   -1,    -1,    -1, ...,   890,   155,   191],
       ...,
       [   -1,    -1,    -1, ...,   145,    21,    47],
       [   -1,    -1,    -1, ..., 13272,    21,    47],
       [   -1,    -1,    -1, ...,    45,   189,    47]])

In [1056]:
test_data = [
    ["情報・通信システム、社会・産業システム、電子装置・システム、建設機械、高機能材料、オートモティブシステム、生活・エコシステム、その他の8の部門から構成されている。売上高9兆3686億円、営業利益7146億円、総従業員数30万7275人は、総合電機最大であり、日本の全業種中でもトヨタ自動車に次ぐ規模の従業員数を誇る巨大企業"],
    ["LSIをはじめとする電気機器を開発・販売する日本の企業。主にパチンコ・パチスロに組み込む、グラフィック・サウンドに関わるLSIの開発・販売に特化したファブレス企業で、直近ではグラフィック・音源・LED制御の機能を1チップに集約したLSI・AG333が主力商品となっている。"],
    ["神奈川県茅ヶ崎市に本社を置く、主に産業・研究機関向け真空装置を製造する企業である。東京証券取引所一部上場。"],
    ["神奈川県横浜市港北区に本社を置く電気計測機器、電気応用機器、電気制御装置の製造、販売を行う企業である。"],
    ["埼玉県さいたま市に本社を置く、電源3社の一角を占める電気機器メーカーである。1938年に、前身である富士電炉工業株式会社が設立され、電気炉、亜酸化銅整流器などの製造を開始した。"]
]

test_sentence_list = [clean_text(s[0]) for s in test_data]
test_sentence_id_vec = convert_sentences_to_ids(test_sentence_list, word_to_id)
test_sentence_id_vec = padding_sentence(test_sentence_id_vec)

for test_sentence_id in test_sentence_id_vec:
    query = np.array(test_sentence_id).reshape(1,len(test_sentence_id_vec[0]))
    result = model.predict(query)
    print(result)
    print(result.argmax())

[[0.00367369 0.00357571 0.02506657 0.08387161 0.01121853 0.00480542
  0.01746091 0.00948998 0.00542732 0.0022201  0.00464994 0.00531497
  0.00465473 0.00964024 0.0172936  0.03042981 0.01864598 0.00890709
  0.02603087 0.01044209 0.03420252 0.01355177 0.00532797 0.0042007
  0.15940301 0.03520033 0.11352297 0.01730782 0.00852554 0.00744745
  0.01757787 0.02892854 0.25198433]]
32
[[0.00374954 0.00345663 0.02603263 0.08249741 0.01102439 0.00465203
  0.01772351 0.00936109 0.00532804 0.00229482 0.00470196 0.0055455
  0.00470697 0.00966898 0.0170799  0.02981348 0.01871423 0.00897993
  0.02516467 0.00991272 0.03570086 0.01268315 0.00534209 0.0041937
  0.1579042  0.03457319 0.1112005  0.01796115 0.00828757 0.00762041
  0.01761159 0.02876833 0.25774485]]
32
[[0.00358255 0.00345697 0.02559888 0.08333034 0.01071802 0.00450204
  0.0170631  0.0094291  0.00541021 0.00223551 0.00474385 0.00545169
  0.00457818 0.00957017 0.01731987 0.03019186 0.01855867 0.00889602
  0.02545623 0.00994841 0.03493635 0.01

In [1057]:
test_sentence_list

['情報・通信システム、社会・産業システム、電子装置・システム、建設機械、高機能材料、オートモティブシステム、生活・エコシステム、その他の8の部門から構成されている。売上高9兆3686億円、営業利益7146億円、総従業員数30万7275人は、総合電機最大であり、日本の全業種中でもトヨタ自動車に次ぐ規模の従業員数を誇る巨大企業',
 'lsiをはじめとする電気機器を開発・販売する日本の企業。主にパチンコ・パチスロに組み込む、グラフィック・サウンドに関わるlsiの開発・販売に特化したファブレス企業で、直近ではグラフィック・音源・led制御の機能を1チップに集約したlsi・ag333が主力商品となっている。',
 '神奈川県茅ヶ崎市に本社を置く、主に産業・研究機関向け真空装置を製造する企業である。東京証券取引所一部上場。',
 '神奈川県横浜市港北区に本社を置く電気計測機器、電気応用機器、電気制御装置の製造、販売を行う企業である。',
 '埼玉県さいたま市に本社を置く、電源3社の一角を占める電気機器メーカーである。1938年に、前身である富士電炉工業株式会社が設立され、電気炉、亜酸化銅整流器などの製造を開始した。']

In [605]:
df = company_pd[["category","category_od"]]
a_df = df[~df.duplicated()]

In [606]:
db = pd.merge(a_df, category, on='category', how='left')

In [607]:
db

,category,category_od,中分類
0,0050,0,水産・農林業
1,1050,1,鉱業
2,2050,2,建設業
3,3050,3,食料品
4,3100,4,繊維製品
5,3150,5,パルプ・紙
6,3200,6,化学
7,3250,7,医薬品
8,3300,8,石油・石炭製品
9,3350,9,ゴム製品
